In [2]:
import pyodbc
import pandas as pd

# Conexão com o banco Firebird
conn_str = (
    "DRIVER=Firebird/InterBase(r) driver;"
    "UID=CONSULTORIA;"
    "PWD=HM#2024!;"
    "DBNAME=mk.rpsolution.com.br/30509:/banco/hmrubber/hmrubber.fdb;"
    "CHARSET=UTF8;"
)

cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Consulta SQL completa (em uma string de múltiplas linhas)
query = """
SELECT 
  N.REGISTRO AS ID_NOTA, N.EMPRESA, N.DATA, N.CLIENTE AS ID_CLIENTE,
  P.RAZAOSOC AS FANTASIA, P.CIDADE, P.ESTADO,
  PV.FANTASIA AS NOME_VEND, CT.CATEGORIA, P.ATIVIDADE, R.REGIAO, Q.EQUIPE,
  T.DESCRICAO AS TIPO_MOV, I.NFE_CFOP AS CFOP,
  EXTRACT(MONTH FROM N.DATA) AS MES,
  EXTRACT(YEAR FROM N.DATA) AS ANO,
  CAST(N.NOTA AS VARCHAR(10)) || '/' || N.SERIE AS NRNOTA,
  I.PRODUTO, M.CODIGO, M.DESCRICAO, M.MARCA, G.GRUPO, S.SUBGRUPO, I.QUANT, L.DESCRICAO AS LOCAL_ESTOQUE,
  CAST((I.TOTMERC/I.QUANT) AS NUMERIC(15,4)) AS VALOR_UNIT, I.TOTMERC,
  -COALESCE(I.NFE_VDESC, 0) AS DESCONTO,
  I.DESCONTO AS PERC_DESC,
  CAST((-I.QUANT*(I.VALOR*(I.DESCONTO/100))) AS NUMERIC(15,2)) AS VALOR_DESC,
  CAST((I.TOTMERC - I.VALORICM - I.VALOR_PIS - I.VALOR_COFINS - I.ICMS_ZF - I.PIS_COFINS_ZF - COALESCE(I.NFE_VDESC,0) - COALESCE(I.NFE_VFRETE,0) - COALESCE(I.NFE_VDESPESAS,0) - COALESCE(I.NFE_VSEGURO,0)) AS NUMERIC(15,2)) AS FAT_LIQUIDO,
  -(
    I.QUANT * CAST(
      (SELECT FIRST 1 COALESCE(PM.VALOR,0) FROM PRECO_MEDIO_DIA PM WHERE PM.ID_MATERIAL = MC.REGISTRO AND PM.DATA BETWEEN '01/01/2010' AND N.DATA ORDER BY PM.DATA DESC)
      AS NUMERIC(15,4)
    )
  ) AS VALOR_CMV,
  -(((I.TOTMERC - I.ICMS_ZF + I.PIS_COFINS_ZF) * I.COMISSAO) / 100) AS VALOR_COMISSAO,
  CAST(COALESCE(I.NFE_VFRETE,0) + COALESCE(I.NFE_VDESPESAS,0) + COALESCE(I.NFE_VSEGURO,0) AS NUMERIC(15,2)) AS DESPESAS,
  -COALESCE((N.VALOR_FRETE_PAGO * (I.TOTMERC / N.TOTMERC)), 0) AS VALOR_FRETE,
  -(COALESCE(F.VALOR, 0) * (I.TOTMERC / N.TOTMERC)) AS VAL_OUTRAS,
  -(I.VALOR_PIS + I.VALOR_COFINS) AS VLR_PIS_COFINS,
  -(I.ICMS_ZF + I.PIS_COFINS_ZF) AS VLR_ZF,
  -COALESCE(I.VALORICM,0) AS VALORICM,
  -COALESCE(I.VALORIPI,0) AS VALORIPI,
  -COALESCE(I.VALOR_ISS,0) AS VALOR_ISS,
  -COALESCE(I.SUBSTRIB,0) AS SUBSTRIB,
  -CAST(((I.TOTMERC + COALESCE(I.NFE_VFRETE,0) + COALESCE(I.NFE_VDESPESAS,0) + COALESCE(I.NFE_VSEGURO,0) - COALESCE(I.NFE_VDESC,0) + I.VALORIPI + I.SUBSTRIB - I.ICMS_ZF - I.PIS_COFINS_ZF) * (N.DESC_COM / 100)) AS NUMERIC(15,2)) AS DESCCOM,
  -(
    COALESCE(I.VALOR_RET_PIS,0) + COALESCE(I.VALOR_RET_COFINS,0) + COALESCE(I.VALOR_RET_CSSL,0) + COALESCE(I.VALOR_IR,0) + COALESCE(I.VALOR_RET_INSS,0) + COALESCE(I.VALOR_RET_ISS,0) + COALESCE(I.VALOR_RET_CPOM,0)
  ) AS RETENCOES,
  I.NFE_VICMSUFREMET, I.NFE_VICMSUFDEST

FROM ITEMNOTA I
INNER JOIN NOTAS N ON N.REGISTRO = I.NOTA
INNER JOIN EMPRESAS E ON E.REGISTRO = N.EMPRESA
INNER JOIN TIPO_VENDA T ON T.REGISTRO = I.ID_TIPONOTA
INNER JOIN CLIENTES C ON (C.PESSOA = N.CLIENTE AND C.EMPRESA = E.TAB_CLIENTES)
INNER JOIN PESSOAS P ON P.CODIGO = N.CLIENTE
INNER JOIN MATERIAIS M ON M.REGISTRO = I.PRODUTO
INNER JOIN MATERIAIS_COMPL MC ON (MC.REG_MATERIAL = I.PRODUTO AND MC.REG_EMPRESA = N.EMPRESA)
INNER JOIN GRUPOS G ON G.REGISTRO = M.GRUPO
INNER JOIN SUBGRUPOS S ON S.REGISTRO = M.SUBGRUPO
INNER JOIN LOCAIS_ESTOQUE L ON L.REGISTRO = I.REG_ESTOQUE
LEFT JOIN PESSOAS PV ON PV.CODIGO = N.VENDEDOR
LEFT JOIN FRETE_COMPRAS F ON F.REG_NFE = N.REGISTRO
LEFT JOIN CATEGORIAS CT ON CT.REGISTRO = C.CATEGORIA
LEFT JOIN VENDEDORES V ON (V.PESSOA = N.VENDEDOR AND V.EMPRESA = 3)
LEFT JOIN VENDAS_REGIAO R ON R.REGISTRO = C.ID_REGIAO
LEFT JOIN EQUIPES Q ON Q.REGISTRO = V.EQUIPE

WHERE N.EMPRESA = 1
  AND N.SITUACAO = 'N'
  AND N.DATA BETWEEN '2025-01-01' AND CURRENT_DATE
  AND I.ID_TIPONOTA IN (4,338,8,232,185,96,302,301,346,65,2,222,340,243,223,341,112,98,226,156,225,339,1,213,211,212,64,218,219,333,69,233,234,330,81)
ORDER BY N.DATA, N.NOTA
"""

# Executando a consulta
df = pd.read_sql(query, cnxn)

print(df)

# Fechando a conexão
cursor.close()
cnxn.close()

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_54184\2089511855.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


    ID_NOTA  EMPRESA        DATA  ID_CLIENTE  \
0     41953        1  2025-01-08        4516   
1     41964        1  2025-01-09       10194   
2     42239        1  2025-01-24       10194   
3     42708        1  2025-02-18         849   
4     42987        1  2025-02-28       10194   
5     43684        1  2025-04-15       10194   
6     44214        1  2025-05-12        3251   
7     44331        1  2025-05-16        6180   
8     44331        1  2025-05-16        6180   
9     44332        1  2025-05-16        6180   
10    44332        1  2025-05-16        6180   
11    44332        1  2025-05-16        6180   
12    44332        1  2025-05-16        6180   
13    44332        1  2025-05-16        6180   
14    44480        1  2025-05-23       12405   
15    44480        1  2025-05-23       12405   
16    44480        1  2025-05-23       12405   
17    44480        1  2025-05-23       12405   
18    44581        1  2025-05-29       11767   
19    44581        1  2025-05-29       1